In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('loans.csv')
df.loc[df.Gender == 'Male', 'Gender'] = 1
df.loc[df.Gender == 'Female', 'Gender'] = 2
df.loc[df.Married == 'No', 'Married'] = 0
df.loc[df.Married == 'Yes', 'Married'] = 1
df.loc[df.Dependents == '3+', 'Dependents'] = 4
df.loc[df.Education == 'Graduate', 'Education'] = 1
df.loc[df.Education == 'Not Graduate', 'Education'] = 0
df.loc[df.Self_Employed == 'No', 'Self_Employed'] = 0
df.loc[df.Self_Employed == 'Yes', 'Self_Employed'] = 1
df.loc[df.Property_Area == 'Urban', 'Property_Area'] = 1
df.loc[df.Property_Area == 'Rural', 'Property_Area'] = 2
df.loc[df.Property_Area == 'Semiurban', 'Property_Area'] = 3
df['Self_Employed'].fillna(0, inplace=True)
df['Dependents'].fillna(0, inplace=True)
df['Dependents'] = pd.to_numeric(df['Dependents'].fillna(0))

x = df[['ApplicantIncome']].values.astype(float)
x_scaled = MinMaxScaler().fit_transform(x)
df[['ApplicantIncome']] = x_scaled

x = df[['CoapplicantIncome']].values.astype(float)
x_scaled = MinMaxScaler().fit_transform(x)
df[['CoapplicantIncome']] = x_scaled

x = df[['LoanAmount']].values.astype(float)
x_scaled = MinMaxScaler().fit_transform(x)
df[['LoanAmount']] = x_scaled

x = df[['Loan_Amount_Term']].values.astype(float)
x_scaled = MinMaxScaler().fit_transform(x)
df[['Loan_Amount_Term']] = x_scaled

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /home/edhiraj/.miniconda/envs/kaggle/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpniwezne0
  JVM stdout: /tmp/tmpniwezne0/h2o_edhiraj_started_from_python.out
  JVM stderr: /tmp/tmpniwezne0/h2o_edhiraj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,2 months and 7 days
H2O cluster name:,H2O_from_python_edhiraj_n863ae
H2O cluster total nodes:,1
H2O cluster free memory:,1.930 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
data = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Input parameters that are going to train
training_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']
# Output parameter train against input parameters
response_column = 'Loan_Status'

In [5]:
# Split data into train and testing
train, test = data.split_frame(ratios=[0.7])

In [6]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=training_columns, y=response_column, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [7]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_20190129_185651_model_2,0.799815,0.474326,0.277737,0.383357,0.146963
GBM_3_AutoML_20190129_185651,0.79548,0.466873,0.28607,0.381628,0.14564
GBM_4_AutoML_20190129_185651,0.795387,0.466198,0.281903,0.381471,0.14552
GBM_1_AutoML_20190129_185651,0.794834,0.472383,0.290237,0.383731,0.14725
XGBoost_1_AutoML_20190129_185651,0.794465,0.450169,0.276368,0.376888,0.142045
GBM_grid_1_AutoML_20190129_185651_model_1,0.793773,0.459896,0.275892,0.381563,0.14559
XGBoost_grid_1_AutoML_20190129_185651_model_1,0.792866,0.452296,0.276368,0.376562,0.141799
GBM_2_AutoML_20190129_185651,0.792251,0.467875,0.275415,0.383139,0.146795
XGBoost_grid_1_AutoML_20190129_185651_model_4,0.787192,0.452854,0.276368,0.37778,0.142717
StackedEnsemble_AllModels_AutoML_20190129_185651,0.78607,0.455847,0.276368,0.379463,0.143992


In [8]:
# The leader model is stored here
aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


In [9]:
preds

predict,N,Y
Y,0.0238944,0.976106
Y,0.071427,0.928573
Y,0.0265313,0.973469
Y,0.48699,0.51301
Y,0.472061,0.527939
Y,0.0835148,0.916485
N,0.912208,0.0877915
N,0.94325,0.0567496
Y,0.0320019,0.967998
Y,0.0362345,0.963766


In [10]:
test

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001006,1,1,0,0,0,0.0300928,0.0565915,0.160637,0.74359,1,1,Y
LP001024,1,1,2,1,0,0.0377242,0.0167999,0.0882779,0.74359,1,1,Y
LP001027,1,1,2,1,0,0.0290662,0.0441596,0.144718,0.74359,1,1,Y
LP001030,1,1,2,1,0,0.0142115,0.0260638,0.0115774,0.230769,1,1,Y
LP001034,1,0,1,0,0,0.0426221,0,0.131693,0.487179,nan,1,Y
LP001046,1,1,1,1,0,0.0717996,0.134999,0.442836,0.74359,1,1,Y
LP001047,1,1,0,0,0,0.030303,0.0458636,0.154848,0.74359,0,3,N
LP001050,nan,1,2,0,0,0.039765,0.0460076,0.149059,0.74359,0,2,N
LP001052,1,1,1,1,0,0.0441187,0.0701994,0.205499,0.74359,nan,3,N
LP001087,2,0,2,1,0,0.0445269,0.0499916,0.160637,0.74359,1,3,Y


In [11]:
aml.leader.r2()

0.9677880183174995